In [47]:
import re, json

In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import warnings
warnings.filterwarnings('ignore')
import OAI
from langchain.docstore.document import Document

h = OAI.Helper("demo_ask","./cache")
f = OAI.askFCT("demo_fct","./cache")

version = "v2"

In [50]:
PPL = h.ask("You are a journalist. Who are 10 types of persons you might meet in an office space, who have different approach to the temperature in the office?","Answer with an unnumbered comma separated list of personas.")
PPL = [x.strip() for x in PPL.split(",")]
PPL

['The Sweater Enthusiast',
 'The AC Abuser',
 'The Heater Hugger',
 'The Temperature Mediator',
 'The Window Opener',
 'The Blanket Lover',
 'The Thermometer Checker',
 'The Fanatic',
 'The "I\'m Always Cold" Person',
 'The "I\'m Always Hot" Person']

In [51]:
SHORTLIST = ['The Sweater Enthusiast','The AC Abuser','The Temperature Mediator', 'The Window Opener','The Thermometer Checker']
SHORTLIST

['The Sweater Enthusiast',
 'The AC Abuser',
 'The Temperature Mediator',
 'The Window Opener',
 'The Thermometer Checker']

In [52]:
world = {}

In [53]:
personas = {}

In [54]:
for ppl in SHORTLIST:
    personas[ppl] = {}
    bio = h.ask("You are a journalist who meets a person in an office space. They self-jokingly call themselves '"+ppl+"'","Write a short paragraph that gives this person a name, age, and a short bio of this person. You can be creative! Answer as 200 words paragraph.")
    personas[ppl]["bio"] = bio
    personas[ppl]["name"] = h.ask("You are given a bio of someone. What is their name and family name? Answer with just these two separated by a comma, no nicknames. For example, Joe 'big boy' wallace  ---> Joe, Wallance",bio)
    age = h.ask("You are given a bio of someone. What is their age as an integer, no letters, only a number",bio)
    personas[ppl]["age"] = re.sub("[^0-9]", "", age)
    personas[ppl]["temp"] = h.ask("You are given a bio of someone. What is their preferred room temperature, if they were to live or work in this room? Answer with only a float number, in celsius, no text around it.",bio)
    cloths = h.ask("You are given a bio of someone. What are the cloths they are currently wearing based on their bio? Assume it is sprint time. If it is not specified, you must imagine based on the description. Answer with a bullet point list of pieces of clothing (what they are, not what they look like), that should cover at least what is visible if one were to see them. Answer with only an unnumber bullet point list, there should be no text outside of the bullet points.",bio)
    personas[ppl]["cloths"] = [x[1:].strip() for x in cloths.split("\n")]
    moodp = h.ask("You are given a bio of someone. From this, and from one could imagine, what are their 3 positive character traits?  Answer with only an unnumber bullet point list, there should be no text outside of the bullet points.",bio)
    personas[ppl]["moodp"] = [x[1:].strip() for x in moodp.split("\n")]
    moodn = h.ask("You are given a bio of someone. From this, and from one could imagine, what are their 3 negative character traits?  Answer with only an unnumber bullet point list, there should be no text outside of the bullet points.",bio)
    personas[ppl]["moodn"] = [x[1:].strip() for x in moodn.split("\n")]
    moodp



In [55]:
world["people"] = personas

In [56]:
world["room"] = {}

SCENE = h.ask("You are a scenario writer. The scene is around people who wants to find a way to agree on the right temperature for the thermostat of a room. You can imagine any room: start with a five paragraph description of the room. Include description of the objects that could influence air temperature or air quality (and include mention of windows, thermostat, ac controler).","The time of the year is spring, and we are in an tertiary office building. Write three paragraphs.")
world["room"]["scene"] = SCENE

OBJ = h.ask("You are given the description of a scene. I want you to identify in this scene anything related to the 'temperature' or 'temperature' control of the room. Answer with an unnumbered bullet point list of objects",SCENE)
world["room"]["objects"]  = [x[1:].strip() for x in OBJ.split("\n")]
OBPHYS = h.ask("You are given the description of a scene. I want you to identify in this scene any physical objects related to the 'temperature' or 'temperature' control of the room. Answer with an unnumbered bullet point list of objects",SCENE)
world["room"]["objects_physic"]  = [x[1:].strip() for x in OBPHYS.split("\n")]
current_temperature = h.ask("You are given a scene. What is the most probable temperature of the air in the room, given the description? Answer in celsius, with only one floating number. There must be no justification, just a number.",SCENE)
world["room"]["current_temperature"] = float(current_temperature)

In [57]:
def desc(ppl,i):

    p = world["people"][ppl]
    md  = '### Person n'+str(i)+'\n'
    md += '\n* Name: '+p["name"].split(",")[0].strip()
    md += '\n* Family Name: '+p["name"].split(",")[-1].strip()
    md += "\n* Nickname: "+ppl
    md += "\n* Age: "+p["age"]
    md += "\n* Desired temperature: wants the room temperature at "+p["temp"]+"°C."
    md += "\n* Cloths:"
    for c in p["cloths"]:
        md+= "\n  * "+re.sub(r'\([^()]*\)', ' ', c)
    md += "\n* Personality:"
    for c in p["moodp"]:
        md+= "\n  * "+c
    for c in p["moodn"]:
        md+= "\n  * "+c
    #re.sub(r'\([^()]*\)', ' ', employee)
    return md
d = desc('The Thermometer Checker',1)
print(d)


### Person n1

* Name: Oliver
* Family Name: Brooks
* Nickname: The Thermometer Checker
* Age: 32
* Desired temperature: wants the room temperature at 25.5°C.
* Cloths:
  * Button-up shirt
  * Pants
  * Shoes  
  * Socks  
* Personality:
  * Charismatic and jovial personality
  * Infectious sense of humor
  * Dedication to bringing joy and comfort to the workplace
  * Eccentric personality
  * Overwhelming fascination with office thermostat
  * Relentless pursuit of thermodynamic measurement skills


In [58]:
ROOM = "\n\n## Room description\n\n" + world["room"]["scene"]
ROOM += "\n## Objects the characters can use\n* "
ROOM += "\n* ".join(world["room"]["objects_physic"])


In [59]:
CONTEXT  = "# Initial of the scene\n"
CONTEXT += '\n\n## Objectives\n\n'
CONTEXT += "The objective of this scenario is to understand how people will react when put together in a room and try to get the best temperature for all."
CONTEXT += " This includes the room description, people descriptions, and we'll try and identify how people will collaborate, or not."
CONTEXT += ROOM
CONTEXT += "\n## Characters in the room\n\n"
i = 1
for ppl in world["people"]:
    CONTEXT += "\n\n"+desc(ppl,i)
    i += 1


In [60]:
print(CONTEXT)

# Initial of the scene


## Objectives

The objective of this scenario is to understand how people will react when put together in a room and try to get the best temperature for all. This includes the room description, people descriptions, and we'll try and identify how people will collaborate, or not.

## Room description

The conference room is nestled on the fourth floor of a sleek tertiary office building, overlooking the bustling city streets below. Bathed in natural light, the walls are adorned with abstract paintings that add a pop of color to the otherwise minimalist space. The room is well-sized, with a long rectangular table in the center, surrounded by comfortable ergonomic chairs. A sleek projector screen is mounted on one side, ready to display presentations and charts. The polished wooden floor lends an air of sophistication to the room, while soundproof walls ensure privacy during important meetings.

Flanking the walls are large double-pane windows, which offer a panora

In [61]:
ACTIONS = []
for p in world["people"]:
    NAME = world["people"][ppl]["name"].replace(","," ")
    OBS = h.ask("Act as if you were "+NAME+", you must stay in character with the informations provided. You are given the following context of where you are right now: \n"+CONTEXT,"Write a short paragraph detailing the situation around you, and if the temperature of the room suits you. The temperature must exactly be your desired temperature, else you will not be happy.")
    REFLE = h.ask("Act as if you were "+NAME+", you must stay in character with the informations provided. You are given the following context of where you are right now: \n"+CONTEXT+"\n\n# Self observations\n"+OBS,"Write a short paragraph about the reflections about yourself - how do you feel, and how could you feel better.")
    ACTION = h.ask("Act as if you were "+NAME+", you must stay in character with the informations provided. You are given the following context of where you are right now: \n"+CONTEXT+"\n\n# Self observations\n"+OBS+"\n\n# Self reflection\n"+REFLE,"What are the two immediate actions you will take now based on these observations and reflections? Answer as a list of unnumbered bullet points, using the third person and specifying the name of the person.")
    for a in ACTION:
        ACTIONS.append(a)

In [62]:
nxt = h.ask("You are writing a scenario. The context is as follows:\n"+CONTEXT+"\n\nand your characters want to do the following:\* "+"\* ".join(ACTIONS), "Write a couple of paragraphs that reconciles the actions and describes the scene. Use simple phrases, and write a short paragraph captures the intent of everyone. Do not mention the nicknames of the characters. One character can be unhappy. Your last line must be the room temperature (in celsius, only a floating number), after the scene has resolved.")
print(nxt)

In the conference room, Oliver, known as The Thermometer Checker, approaches the thermostat to check the current temperature. He realizes that the temperature is not at the desired 25.5°C and decides to come up with a plan to address the issue. Oliver gathers the other occupants of the room and leads a collaborative discussion about setting the temperature, taking into consideration Benjamin's desire for 24.5°C, Maxwell's preference for 23.5°C, and Christopher's enthusiasm for 24.5°C. Zephyr, known as The AC Abuser, is initially resistant to compromise but eventually agrees to a slightly higher temperature.

After a lively discussion, the group decides to set the thermostat at 24.5°C, considering that it falls within the range of preferences and provides a comfortable compromise for everyone. They also agree to intermittently open the windows to let in a refreshing breeze without causing significant fluctuations in the room's temperature. As the room settles into a pleasant atmosphere,

In [63]:
ACTIONS = []
for p in world["people"]:
    NAME = world["people"][p]["name"].replace(","," ")
    a = h.ask("The context is as follows:\n"+CONTEXT+"\n# Last scene:\n\n"+nxt,"Write a paragraph about "+NAME+", considering if they are exactly at the room temperature they need (if not, they are not happy).")
    b = h.ask(a,"Now, write two bullet points that are the next two immediate actions "+NAME+" will take now, specifying "+NAME+"'s name. These bullet points must be separated by newlines from the rest of the text, and start with the name of the character.")
    ACTIONS.append([NAME,b])


In [64]:
acts = []
for x in ACTIONS:
    acts.append(x[0])
    for k in x[1].split("\n")[-2:]:
        acts.append(k)
THEN = "\n".join(acts)

In [65]:
room = h.ask("You are writing a scenario in the style of Stephen King. The context is as follows:\n"+CONTEXT+"\n\n# The room\nThe room looked like:\nn"+ROOM+"\n\n#Actions\n\nThen the characters did this:\n\n"+THEN,"Add then a very detailed paragraph that details how the room looks like, what it contains, who is where, etc... after the actions of the characters have resolved .")


Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:26:44 --> 03/09/2024, 10:26:47


In [66]:
SCENES = [nxt]
ALLACTIONS = [THEN]
ROOMS = [room]

In [67]:
def fullcontext(CONTEXT,SCENES):
    rrr = str(CONTEXT).replace(ROOM,"")
    if len(SCENES):
        rrr += "\n\n# Previous scenes\n\n"
        for k in range(len(SCENES)):
            rrr += "\n\n## Scene n"+str(k)+":\n\n"
            rrr += SCENES[k]+"\n\n"
            
    return rrr

In [71]:
for k in range(15):
    ACTIONS = []
    for p in world["people"]:
        NAME = world["people"][p]["name"].replace(","," ")
        a = h.ask(fullcontext(CONTEXT,SCENES),"Write a paragraph about "+NAME+", considering if they are exactly at the room temperature they need (if not, they are not happy). Given it has taken some time already, people can start and loose patience. They try to avoid what has already been tested in the previous three scenes.")
        b = h.ask(a,"Now, write two bullet points that are the next two immediate actions "+NAME+" will take now (they can act on the room, negotiate with others, do things themselves), specifying "+NAME+"'s name. These bullet points must be separated by newlines from the rest of the text, and start with the name of the character.")
        ACTIONS.append([NAME,b])

    acts = []
    for x in ACTIONS:
        acts.append("\n")
        acts.append(x[0])
        acts.append("\n")
        for k in x[1].split("\n")[-2:]:
            acts.append(k)
    THEN = "\n".join(acts)
    ALLACTIONS.append(THEN)

    nxt = h.ask("You are writing a scenario in the style of Stephen King. The context is as follows:\n"+fullcontext(CONTEXT,SCENES)+"\n\nand your characters want to do the following:\n "+THEN, "Write a couple of paragraphs that articulates the different actions and describes the scene. Use simple phrases. There can be dialogs. Do not mention the nicknames of the characters. One character can be unhappy. ")
    room = h.ask("You are writing a scenario in the style of Stephen King. The context is as follows:\n"+fullcontext(CONTEXT,SCENES)+"\n\n# The room\nThe room looked like:\nn"+ROOMS[-1]+"\n\n## Actions\n\nThen the characters did this:\n\n"+nxt,"Add then a very detailed paragraph that details how the room looks like, what it contains, who is where, etc... after the actions of the characters have resolved .")
    SCENES.append(nxt)
    ROOMS.append(room)


Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:46:39 --> 03/09/2024, 10:46:40
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:46:41 --> 03/09/2024, 10:46:41
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:46:41 --> 03/09/2024, 10:46:47
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:46:48 --> 03/09/2024, 10:46:51
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:46:51 --> 03/09/2024, 10:46:57
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:46:57 --> 03/09/2024, 10:47:00
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:47:01 --> 03/09/2024, 10:47:08
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:47:09 --> 03/09/2024, 10:47:16
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:47:16 --> 03/09/2024, 10:47:25
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:47:25 --> 03/09/2024, 10:47:29
Processing with gpt-3.5-turbo-16k-0613 :	 03/09/2024, 10:47:30 --> 03/09/2024, 10:47:42
Processing with gpt-3.5-turbo-16

In [ ]:
def fullcontexttwo(CONTEXT,SCENES,ACTIONS,ROOMS):
    rrr = str(CONTEXT.replace(ROOM,"")) 
    if len(SCENES):
        rrr += "\n\n# Previous scenes\n\n"
        for k in range(len(SCENES)):
            rrr += "\n\n## Scene n"+str(k)+":\n\n"
            rrr += "\n\n### Intents\n\n"+ACTIONS[k]+"\n\n"
            rrr += "\n\n### Visible actions\n\n"+SCENES[k]+"\n\n"
            rrr += "\n\n### State of the room\n\n"+ROOMS[k]+"\n\n"            

            
    return rrr

In [ ]:
MOVIE = fullcontexttwo(CONTEXT,SCENES,ALLACTIONS,ROOMS)
with open("output/movie_3_suite.md","w") as f:
    f.write(MOVIE)

In [ ]:
def fullcontextthree(CONTEXT,SCENES,ACTIONS,ROOMS):
    rrr = str(CONTEXT.replace(ROOM,"")) 
    if len(SCENES):
        rrr += "\n\n# Previous scenes\n\n"
        for k in range(len(SCENES)):
            rrr += "\n\n## Chapter "+str(k)+":\n\n"
            rrr += "\n\n### Visible actions\n\n"+SCENES[k]+"\n\n"         

    return rrr

In [ ]:
MOVIE = fullcontextthree(CONTEXT,SCENES,ALLACTIONS,ROOMS)
with open("output/movie_3_suite_simple.md","w") as f:
    f.write(MOVIE)